In [1]:
import pandas as pd
import re

### Import

In [2]:
# Importing the signature key in two parts

df1 = pd.read_csv('data/bhr1.csv', error_bad_lines=False, sep=';')
columns = df1.columns
df2 = pd.read_csv('data/bhr2.csv', names = columns, error_bad_lines=False, sep=';')
df = pd.concat([df1, df2], ignore_index=True)


C:\Users\hanna\AppData\Local\Temp/ipykernel_21292/440638329.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df1 = pd.read_csv('data/bhr1.csv', error_bad_lines=False, sep=';')
C:\Users\hanna\AppData\Local\Temp/ipykernel_21292/440638329.py:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df2 = pd.read_csv('data/bhr2.csv', names = columns, error_bad_lines=False, sep=';')


## Cleaning

In [13]:
df.drop('vw', axis=1, inplace=True)

#### Text 

In [14]:
# getting rid of the ... in the texts
df["text"] = df["text"].str.replace(r'\.{3,}', '')

C:\Users\hanna\AppData\Local\Temp/ipykernel_21292/1185189639.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["text"] = df["text"].str.replace(r'\.{3,}', '')


In [11]:
df[df.text.str.startswith('Chiesa')]

,lev,sys,numbis,text,vw
3554,4,Dn 1010,NaN,Chiesa Addolorata,NaN
3556,4,Dn 1030,NaN,Chiesa Angeli Custodi,NaN
3557,4,Dn 1040,NaN,Chiesa Bambino Gesù,NaN
3558,4,Dn 1050,NaN,Chiesa Cappuccini,NaN
3560,4,Dn 1070,NaN,Chiesa Corpus Domini,NaN
...,...,...,...,...,...
3855,4,Dn 4700,NaN,Chiesa Trinità dei Pellegrini,NaN
3856,4,Dn 4710,NaN,Chiesa Trinità degli Scozzesi,NaN
3857,4,Dn 4720,NaN,Chiesa Trinità degli Spagnuoli,NaN
3858,4,Dn 4730,NaN,Chiesa Valdese (Piazza Cavour),NaN


#### Backreferencing

In [15]:
# replace idem (=...)

backreference = re.compile(r"idem\s*\(\s*=\s*(.*)\s*\)")

df["backreference"] = df["text"].str.extract(backreference, expand=False)

# Replaces the backreference with the first captured group
df["text"] = df["text"].str.replace(backreference, r"\1")



In [16]:
# find the remaining idems

idem = re.compile(r'\bidem\s*[,.]?\s*')

idem_df = df[df["text"].str.contains(idem)]

In [17]:
# replace the idem without backreference

for i, row in idem_df.iterrows():
    # loop over rows in the df DataFrame above the current row
    for j in range(i-1, -1, -1):

        if not idem.search(df.loc[j, "text"]):

            reference = df.iloc[j].text.split(',')[0]
            
            #replace idem with the actual backreference
            substring = df.iloc[i].text

            modified_substring = idem.sub(f'{reference} ', substring)

            df.loc[i, "text"] = modified_substring
            break



In [18]:
df[df.text.str.startswith('Chiesa')]

,lev,sys,numbis,text,backreference
3554,4,Dn 1010,NaN,Chiesa Addolorata,NaN
3556,4,Dn 1030,NaN,Chiesa Angeli Custodi,NaN
3557,4,Dn 1040,NaN,Chiesa Bambino Gesù,NaN
3558,4,Dn 1050,NaN,Chiesa Cappuccini,NaN
3560,4,Dn 1070,NaN,Chiesa Corpus Domini,NaN
...,...,...,...,...,...
3855,4,Dn 4700,NaN,Chiesa Trinità dei Pellegrini,NaN
3856,4,Dn 4710,NaN,Chiesa Trinità degli Scozzesi,NaN
3857,4,Dn 4720,NaN,Chiesa Trinità degli Spagnuoli,NaN
3858,4,Dn 4730,NaN,Chiesa Valdese (Piazza Cavour),NaN


#### Level 1, 2, 3 Texts

In [19]:
# Level 3 text

df['text_3'] = ''

indices = df[df.lev == 3].index

for i in range(len(indices) - 1): 

    start = indices[i]
    end = indices[i+1] 

    df.loc[start + 1 :end - 1, 'text_3'] = df.iloc[start].text

#handle the last case seperately 
if len(indices) > 0: 
    start = indices[-1]
    end = len(df)
    df.loc[start + 1 :end - 1, 'text_3'] = df.iloc[start].text


In [20]:
# Level 2 text 

df['text_2'] = ''

indices = df[df.lev == 2].index

for i in range(len(indices) - 1): 

    start = indices[i]
    end = indices[i+1] 

    df.loc[start + 1 :end - 1, 'text_2'] = df.iloc[start].text

#handle the last case seperately 
if len(indices) > 0: 
    start = indices[-1]
    end = len(df)
    df.loc[start + 1 :end - 1, 'text_2'] = df.iloc[start].text

In [21]:
# Level 1 text 

df['text_1'] = ''

indices = df[df.lev == 1].index

for i in range(len(indices) - 1): 

    start = indices[i]
    end = indices[i+1] 

    df.loc[start + 1 :end - 1, 'text_1'] = df.iloc[start].text


#handle the last case seperately 
if len(indices) > 0: 
    start = indices[-1]
    end = len(df)
    df.loc[start + 1 :end - 1, 'text_1'] = df.iloc[start].text

In [22]:
df[df.text.str.startswith('Chiesa')]

,lev,sys,numbis,text,backreference,text_3,text_2,text_1
3554,4,Dn 1010,NaN,Chiesa Addolorata,NaN,"Klöster, Kirchen, Oratorien, Kapellen und Koll...",Einzelne sakrale Gebäude,Topographie Rom
3556,4,Dn 1030,NaN,Chiesa Angeli Custodi,NaN,"Klöster, Kirchen, Oratorien, Kapellen und Koll...",Einzelne sakrale Gebäude,Topographie Rom
3557,4,Dn 1040,NaN,Chiesa Bambino Gesù,NaN,"Klöster, Kirchen, Oratorien, Kapellen und Koll...",Einzelne sakrale Gebäude,Topographie Rom
3558,4,Dn 1050,NaN,Chiesa Cappuccini,NaN,"Klöster, Kirchen, Oratorien, Kapellen und Koll...",Einzelne sakrale Gebäude,Topographie Rom
3560,4,Dn 1070,NaN,Chiesa Corpus Domini,NaN,"Klöster, Kirchen, Oratorien, Kapellen und Koll...",Einzelne sakrale Gebäude,Topographie Rom
...,...,...,...,...,...,...,...,...
3855,4,Dn 4700,NaN,Chiesa Trinità dei Pellegrini,NaN,"Klöster, Kirchen, Oratorien, Kapellen und Koll...",Einzelne sakrale Gebäude,Topographie Rom
3856,4,Dn 4710,NaN,Chiesa Trinità degli Scozzesi,NaN,"Klöster, Kirchen, Oratorien, Kapellen und Koll...",Einzelne sakrale Gebäude,Topographie Rom
3857,4,Dn 4720,NaN,Chiesa Trinità degli Spagnuoli,NaN,"Klöster, Kirchen, Oratorien, Kapellen und Koll...",Einzelne sakrale Gebäude,Topographie Rom
3858,4,Dn 4730,NaN,Chiesa Valdese (Piazza Cavour),NaN,"Klöster, Kirchen, Oratorien, Kapellen und Koll...",Einzelne sakrale Gebäude,Topographie Rom


#### Ranges

In [12]:
# Appending all the ranges as rows, to make lookup operations easier later

df.numbis = df.numbis.fillna('')
range_rows = df[df.numbis.str.isdigit()]

for i, row in range_rows.iterrows():

    start = int(re.findall(r'\d+', row.sys)[0])
    end = int(row.numbis)

    for j in range(start, end): 
        new_row = row.copy()
        new_row['sys'] = new_row['sys'].replace(str(start), str(j+1))

        df = df.append(new_row, ignore_index=True)


C:\Users\hanna\AppData\Local\Temp/ipykernel_24324/1229544586.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


# Export 

In [14]:
df.to_csv('data/csv/signatures.csv', index=False)

## Data Merging

In [272]:
# Importing the signatures and inventory date

path = 'data/csv/hertz.csv'

df_hertz = pd.read_csv(path)

df_hertz.signature.fillna('', inplace=True)


In [292]:
# Extracting signature with the format Xx 0000


df_hertz[['signature_l', 'signature_n']] = df_hertz.signature.str.extract(r'^([a-zA-Z]{2}) (\d+)', expand=False)


df_hertz['signature_clean'] = df_hertz[['signature_l', 'signature_n']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)


# Merging on the signatures

merged = df_hertz.merge(df, left_on='signature_clean', right_on='sys')

# Export

export_columns = ['id', 'signature', 'inventory_nr', 'text', 'text_3', 'text_2', 'text_1']

merged[export_columns].to_csv('data/csv/hertz_signatures.csv')